# Create a new Universe merging two different systems using MDAnalysis


In [1]:
import MDAnalysis as mda
import numpy as np
import MDAnalysis.transformations
import nglview as nv

The first thing to do is load the structure one to create the universe **u1**. The structure can be visualised with NGLView. 

In [2]:
u1 = mda.Universe("system1.gro")
sol_view = nv.show_mdanalysis(u1)
sol_view.add_representation('ball+stick', selection='all')
sol_view.center()
sol_view


NGLWidget()

Load the structure two to create the universe **u2**

In [4]:
u2 = mda.Universe("system2.gro")
sol_view = nv.show_mdanalysis(u2)
sol_view.add_representation('ball+stick', selection='all')
sol_view.center()
sol_view



NGLWidget()

Now we have to decide across what axis we want join our two systems, the options are: **axis = 0** for $x$ axis, **axis = 1** for $y$ and  **axis = 2** for $z$). The gap between them has to be indicated in Angstroms. The idea is translate the positions of the structure two to merge it correctly (without overlaping) with the structure one. 

In [6]:
axis = 2

gap = 10

u1 = mda.Universe("system1.gro")
u2 = mda.Universe("system2.gro")

atoms1 = u1.select_atoms('resname *')
posiciones1 = atoms1.positions
centroid1 = atoms1.centroid()

atoms2 = u2.select_atoms('resname *')
posiciones2 = atoms2.positions
centroid2 = atoms2.centroid()

delta = centroid1 - centroid2
delta[axis] = centroid1[axis]*2 + gap

workflow = [MDAnalysis.transformations.translate(delta)]
u2 = mda.Universe("system2.gro", transformations=workflow)

cell_size = u1.dimensions
cell_size[axis] = u1.dimensions[axis] + u2.dimensions[axis] + gap

u3 = mda.Merge(u1.atoms, u2.atoms)
u3.dimensions = cell_size

sol_view = nv.show_mdanalysis(u3)
sol_view.add_representation('ball+stick', selection='all')
sol_view.center()
sol_view


NGLWidget()

Finally we can write the coordinates of the **u3** universe to a file **out.gro** with GRO format

In [7]:
u3.atoms.write('out.gro')